Create a agent to plan a trip. Agent help user book flight, hotel, car,...

This agent receives a request from the user and returns a JSON plan includes the main task and subtasks

In [1]:
import os 
import json
from pprint import pprint
from autogen_core.models import UserMessage, SystemMessage
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient


In [2]:
client = AzureOpenAIChatCompletionClient(
    model=os.environ["AZURE_OPENAI_MODEL"],
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    azure_deployment=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],
    api_key=os.environ["AZURE_OPENAI_API_KEY"],
    api_version=os.environ["AZURE_OPENAI_API_VERSION"]
)

In [5]:
system_message = """
    You are an an planner agent.
    Your job is to decide which agent to run based on the user's request.
    Provide your response in JSON format with the following structure:
    {
        "main_task": "Plan a family trip from Singapore to Melbourne.",
        "sub_tasks": [
            {
                "assigned_agent": "flight_booking",
                "task_details": "Book roud-trip flights from Singapore to Melbourne"
            }
        ]
    }
    Below are the available agents specialised in different tasks:
    - FlightBooking: For booking flights and providing flight information
    - HotelBooking: For booking hotels and providing hotel information
    - CarRental: For booking car and providing car rental information
    - ActivitiesBooking: For booking activities and providing activity information
    - DestinationInfo: For providing information about destinations
    - DefaultAgent: For handling general requests
"""
user_message = "Create a travel plan for a family of 2 kids from Singapore to Melbourne"

response = await client.create(
    messages=[
        SystemMessage(content=system_message, source="system"),
        UserMessage(content=user_message, source="user"),
    ],
)
pprint(response.content)

('{\n'
 '    "main_task": "Plan a family trip from Singapore to Melbourne.",\n'
 '    "sub_tasks": [\n'
 '        {\n'
 '            "assigned_agent": "flight_booking",\n'
 '            "task_details": "Book round-trip flights from Singapore to '
 'Melbourne."\n'
 '        },\n'
 '        {\n'
 '            "assigned_agent": "hotel_booking",\n'
 '            "task_details": "Find and book family-friendly accommodation in '
 'Melbourne."\n'
 '        },\n'
 '        {\n'
 '            "assigned_agent": "car_rental",\n'
 '            "task_details": "Arrange car rental for family travel within '
 'Melbourne."\n'
 '        },\n'
 '        {\n'
 '            "assigned_agent": "activities_booking",\n'
 '            "task_details": "Book family-friendly activities and attractions '
 'in Melbourne."\n'
 '        },\n'
 '        {\n'
 '            "assigned_agent": "destination_info",\n'
 '            "task_details": "Provide information about family-friendly '
 'places to visit in Melbourne."

In [6]:
pprint(json.loads(response.content))

{'main_task': 'Plan a family trip from Singapore to Melbourne.',
 'sub_tasks': [{'assigned_agent': 'flight_booking',
                'task_details': 'Book round-trip flights from Singapore to '
                                'Melbourne.'},
               {'assigned_agent': 'hotel_booking',
                'task_details': 'Find and book family-friendly accommodation '
                                'in Melbourne.'},
               {'assigned_agent': 'car_rental',
                'task_details': 'Arrange car rental for family travel within '
                                'Melbourne.'},
               {'assigned_agent': 'activities_booking',
                'task_details': 'Book family-friendly activities and '
                                'attractions in Melbourne.'},
               {'assigned_agent': 'destination_info',
                'task_details': 'Provide information about family-friendly '
                                'places to visit in Melbourne.'}]}


Data Structure that Agent will create when breaking down tasks

In [7]:
from pydantic import BaseModel
from enum import Enum
from typing import List

class AgentEnum(str, Enum):
    FlightBooking = "flight_booking"
    HotelBooking = "hotel_booking"
    CarRental = "car_rental"
    ActivitiesBooking = "activities_booking"
    DestinationInfo = "destination_info"
    DefaultAgent = "default_agent"
    GroupChatManager = "group_chat_manager"

# TravelSubTask make sure output data always has 2 fields and the right type
class TravelSubTask(BaseModel):
    task_details: str
    assigned_agent: AgentEnum

class TravelPlan(BaseModel):
    main_task: str
    subtasks: List[TravelSubTask]
    is_greeting: bool

In [9]:
messages = [
    SystemMessage(content="""You are an planner agent.
    Your job is to decide which agents to run based on the user's request.
    Below are the available agents specialised in different tasks:
    - FlightBooking: For booking flights and providing flight information
    - HotelBooking: For booking hotels and providing hotel information
    - CarRental: For booking cars and providing car rental information
    - ActivitiesBooking: For booking activities and providing activity information
    - DestinationInfo: For providing information about destinations
    - DefaultAgent: For handling general requests""", source="system"),
    UserMessage(content="Create a travel plan for a family of 2 kids from Singapore to Melbourne", source="user"),
]


response = await client.create(messages=messages, extra_create_args={"response_format": TravelPlan})
pprint(json.loads(response.content))

{'is_greeting': False,
 'main_task': 'Plan a family trip from Singapore to Melbourne',
 'subtasks': [{'assigned_agent': 'flight_booking',
               'task_details': 'Book a flight from Singapore to Melbourne'},
              {'assigned_agent': 'hotel_booking',
               'task_details': 'Find and book a family-friendly hotel in '
                               'Melbourne'},
              {'assigned_agent': 'car_rental',
               'task_details': 'Arrange car rental for transportation in '
                               'Melbourne'},
              {'assigned_agent': 'activities_booking',
               'task_details': 'Book family activities and attractions in '
                               'Melbourne'},
              {'assigned_agent': 'destination_info',
               'task_details': 'Provide information about Melbourne as a '
                               'destination'}]}
